In [1]:
import stata_setup
stata_setup.config("C:/Program Files/Stata17/", "mp")


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user 4-core  perpetual
Serial number: 501706303466
  Licensed to: David Tomas Jacho-Chavez
               Emory University

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000; see help set_maxvar.


## Double Selection Lasso

The Double Selection Lasso is implemented by the ```dsregress``` Stata command. It'll utilize a 'plugin' value for the $\lambda$s parameters by default, but they can also use standard cross-validation instead.

In [2]:
%%stata
use https://www.stata-press.com/data/r17/breathe, clear
quietly do https://www.stata-press.com/data/r17/no2
quietly dsregress react no2_class, controls($cc i.($fc)) rseed(52)
estimates store ds_plugin
quietly dsregress react no2_class, controls($cc i.($fc)) selection(cv) rseed(52)
estimates store ds_cv


. use https://www.stata-press.com/data/r17/breathe, clear
(Nitrogen dioxide and attention)

. quietly do https://www.stata-press.com/data/r17/no2

. quietly dsregress react no2_class, controls($cc i.($fc)) rseed(52)

. estimates store ds_plugin

. quietly dsregress react no2_class, controls($cc i.($fc)) selection(cv) rseed(
> 52)

. estimates store ds_cv

. 


In [3]:
%%stata
#delimit ;
lassocoef (ds_plugin, for(react)) (ds_cv, for(react))
          (ds_plugin, for(no2_class)) (ds_cv, for(no2_class));
#delimit cr


. #delimit ;
delimiter now ;
. lassocoef (ds_plugin, for(react)) (ds_cv, for(react))
>           (ds_plugin, for(no2_class)) (ds_cv, for(no2_class));

--------------------------------------------------------------
                  | ds_plugin   ds_cv    ds_plugin    ds_cv   
                  |   react     react    no2_class  no2_class 
------------------+-------------------------------------------
              age |     x         x                     x     
            0.sex |     x         x     
                  |
            grade |
             2nd  |     x         x     
             4th  |     x         x     
             3rd  |                                     x     
                  |
       feducation |
      University  |     x         x                     x     
        <Primary  |               x                     x     
         Primary  |               x     
                  |
             age0 |               x     
         sev_home |               x    

In [4]:
%%stata
estimates restore ds_cv
dsregress


. estimates restore ds_cv
(results ds_cv are active now)

. dsregress

Double-selection linear model         Number of obs               =      1,036
                                      Number of controls          =         32
                                      Number of selected controls =         24
                                      Wald chi2(1)                =      24.17
                                      Prob > chi2                 =     0.0000

------------------------------------------------------------------------------
             |               Robust
       react | Coefficient  std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
   no2_class |   2.405792    .489301     4.92   0.000      1.44678    3.364804
------------------------------------------------------------------------------
Note: Chi-squared test is a Wald test of the coefficients of the variables
      of interest jointly 